In [1]:
# Install Hugging Face libraries
!pip install -U -q peft==0.15.2 transformers==4.51.3 accelerate==1.7.0 trl==0.17.0 bitsandbytes==0.45.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers
import peft
import accelerate
import trl
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("accelerate:", accelerate.__version__)
print("trl:", trl.__version__)
# transformers: 4.51.3
# peft: 0.15.2
# accelerate: 1.6.0
# trl: 0.17.0

transformers: 4.51.3
peft: 0.15.2
accelerate: 1.7.0
trl: 0.17.0


In [3]:
import subprocess
print("bitsandbytes:", subprocess.getoutput("pip show bitsandbytes | grep Version"))
# bitsandbytes: Version: 0.45.5

bitsandbytes: Version: 0.45.5


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
import torch

### 🔍 Note: Loading a Quantized Model

To reduce memory usage and accelerate loading, we use 4-bit quantization with [bitsandbytes](https://github.com/TimDettmers/bitsandbytes). This enables efficient model handling during preference optimization or evaluation, especially on limited hardware.

> ⚠️ This setup is optional and does not alter the DPO algorithm itself.


In [5]:
# ✅ Load quantized model with bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

### 🧩 Applying LoRA to a Quantized Model

We use PEFT via LoRA to fine-tune the quantized base model. This allows us to update a small number of trainable parameters while keeping the rest of the model frozen, significantly reducing memory usage and training time.

Note: The `target_modules` argument must match the architecture of the base model. Common targets include attention and projection layers such as `"c_attn"` and `"c_proj"`.

### 🧩 Applying LoRA to Qwen2 or Similar Architectures

We apply LoRA to the quantized model for parameter-efficient fine-tuning. Since Qwen2 follows a LLaMA-style transformer architecture, we must specify all key projection layers for LoRA adaptation.

> 🔧 The `target_modules` must match the model’s architecture. Incorrect targeting will silently result in no trainable parameters or failed adaptation.


Since the base model is Qwen2, we must adapt the LoRA `target_modules` to match the internal architecture. Attempting to use default names like `"c_proj"` or `"c_attn"` will result in a `ValueError`.

To discover valid modules:

In [6]:
for name, module in model.named_modules():
    if 'linear' in name or 'proj' in name:
        print(name)

model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

Function to check target models to add (Optional)

In [7]:
def suggest_lora_target_modules(model, keywords=("proj", "linear"), verbose=True):
    """
    Suggest target modules for LoRA adaptation based on common naming patterns.

    Args:
        model: The transformer model to inspect.
        keywords: Tuple of substrings to match in module names.
        verbose: If True, prints the suggested module names.

    Returns:
        List of matching module names.
    """
    candidates = []
    for name, module in model.named_modules():
        if any(kw in name.lower() for kw in keywords):
            candidates.append(name)
    if verbose:
        print("🔍 Suggested target_modules for LoRA:\n", candidates)
    return candidates

Target Modelues for Qwen models

In [8]:
target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj"
]

In [9]:
# Call this before applying LoRA
target_modules = suggest_lora_target_modules(model)

🔍 Suggested target_modules for LoRA:
 ['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.0.mlp.gate_proj', 'model.layers.0.mlp.up_proj', 'model.layers.0.mlp.down_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.1.self_attn.o_proj', 'model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.self_attn.q_proj', 'model.layers.2.self_attn.k_proj', 'model.layers.2.self_attn.v_proj', 'model.layers.2.self_attn.o_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.3.self_attn.q_proj', 'model.layers.3.self_attn.k_proj', 'model.layers.3.self_attn.v_proj', 'model.layers.3.self_attn.o_proj', 'model.layers.3.mlp.gate_proj', 'model.layers.3.mlp.up_proj', 'model.layers.3.mlp.down_proj', 'model.layers.4.self_attn

In [10]:
print(target_modules)

['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.0.mlp.gate_proj', 'model.layers.0.mlp.up_proj', 'model.layers.0.mlp.down_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.1.self_attn.o_proj', 'model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.self_attn.q_proj', 'model.layers.2.self_attn.k_proj', 'model.layers.2.self_attn.v_proj', 'model.layers.2.self_attn.o_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.3.self_attn.q_proj', 'model.layers.3.self_attn.k_proj', 'model.layers.3.self_attn.v_proj', 'model.layers.3.self_attn.o_proj', 'model.layers.3.mlp.gate_proj', 'model.layers.3.mlp.up_proj', 'model.layers.3.mlp.down_proj', 'model.layers.4.self_attn.q_proj', 'model.layers.4.self_attn.k_

In [11]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

In [12]:
dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train[:10000]")


def preprocess_function(examples):
    # Combine chosen/rejected into one string for now (for tokenizer sanity check)
    return {
        "prompt": examples["chosen"],
        "chosen": examples["chosen"],
        "rejected": examples["rejected"],
    }

dataset = dataset.map(preprocess_function, batched=True)


README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Split into training and eval dataset

In [13]:
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

Preprocessing using tokenizer

In [14]:
from transformers import AutoTokenizer

# Load tokenizer for the base model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

# Preprocessing function
def preprocess_function(examples):
    input_text = [f"Chosen: {ch} Rejected: {rej}" for ch, rej in zip(examples['chosen'], examples['rejected'])]
    return tokenizer(input_text, padding=True, truncation=True, max_length=512)

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Setting Up configurations and trainer for DPO

In [15]:
# ✅ DPO Config
dpo_config = DPOConfig(
    output_dir="qwen2-dpo-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=5,
    report_to="none",  # disables wandb
    fp16=True          # if supported by GPU
)

In [16]:
# ✅ DPO Trainer (pass the tokenizer as processing_class)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer # <-- Pass tokenizer here
)


Extracting prompt in train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Train/Fine-tune

In [17]:
trainer.train()

Step,Training Loss
5,0.692500
10,0.694300
15,0.692200
20,0.689800
25,0.689800
30,0.687100
35,0.687600
40,0.680900
45,0.677900
50,0.677900


TrainOutput(global_step=2000, training_loss=0.275327322550118, metrics={'train_runtime': 3052.679, 'train_samples_per_second': 2.621, 'train_steps_per_second': 0.655, 'total_flos': 0.0, 'train_loss': 0.275327322550118, 'epoch': 1.0})

In [18]:
trainer.save_model()

In [19]:
from peft import PeftModel

# # Cast to base class if needed (optional)
# if not isinstance(model, PeftModel):
#     print("Warning: model is not a PEFT model")

model.save_pretrained("qwen2-lora-adapter")
tokenizer.save_pretrained("qwen2-lora-adapter")

('qwen2-lora-adapter/tokenizer_config.json',
 'qwen2-lora-adapter/special_tokens_map.json',
 'qwen2-lora-adapter/vocab.json',
 'qwen2-lora-adapter/merges.txt',
 'qwen2-lora-adapter/added_tokens.json',
 'qwen2-lora-adapter/tokenizer.json')

In [20]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.5068392753601074, 'eval_runtime': 138.7124, 'eval_samples_per_second': 14.418, 'eval_steps_per_second': 1.802, 'eval_rewards/chosen': -0.38178980350494385, 'eval_rewards/rejected': -1.2358617782592773, 'eval_rewards/accuracies': 0.5540000200271606, 'eval_rewards/margins': 0.854072093963623, 'eval_logps/chosen': -511.09881591796875, 'eval_logps/rejected': -534.92919921875, 'eval_logits/chosen': -2.7226788997650146, 'eval_logits/rejected': -2.6275696754455566, 'epoch': 1.0}


In [21]:
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [22]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Set Up Trainer

In [23]:
trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer
)

Extracting prompt in train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.4954094886779785, 'eval_model_preparation_time': 0.0054, 'eval_runtime': 138.6141, 'eval_samples_per_second': 14.429, 'eval_steps_per_second': 1.804, 'eval_rewards/chosen': -0.3787333071231842, 'eval_rewards/rejected': -1.2847709655761719, 'eval_rewards/accuracies': 0.5770000219345093, 'eval_rewards/margins': 0.9060377478599548, 'eval_logps/chosen': -518.4610595703125, 'eval_logps/rejected': -537.01904296875, 'eval_logits/chosen': nan, 'eval_logits/rejected': -2.649744987487793}


In [25]:
from transformers import get_scheduler

# Set a lower learning rate to avoid overshooting
dpo_config.learning_rate = 5e-5  # Try lower than your current lr
dpo_config.num_train_epochs = 3  # Try training for 3 epochs or more
dpo_config.lr_scheduler_type = "linear"
dpo_config.per_device_train_batch_size = 4  # Or try 8, 16 depending on your memory constraints
dpo_config.weight_decay = 0.01  # Common default for weight decay

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer,  # Pass tokenizer here
)
trainer.train()

Applying chat template to train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
5,0.262200
10,0.319400
15,0.291100
20,0.427200
25,0.411700
30,0.210100
35,0.214900
40,0.261400
45,0.388800
50,0.326800


KeyboardInterrupt: 

#### Inference and Evaluation

In [26]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.4296061396598816, 'eval_rewards/chosen': -2.217827320098877, 'eval_rewards/rejected': -5.833990573883057, 'eval_rewards/accuracies': 0.6255000233650208, 'eval_rewards/margins': 3.6161632537841797, 'eval_logps/chosen': -536.8519897460938, 'eval_logps/rejected': -582.5112915039062, 'eval_logits/chosen': nan, 'eval_logits/rejected': -2.75342059135437}


 The training loss hovers mostly between 0.1 and 0.3, with a mild downward trend. This suggests the model is learning, albeit slowly. A stable but noisy loss like this is not uncommon for DPO-style fine-tuning, especially if using small batch sizes.
 Learning Rate:
For DPOTrainer, 5e-5 is reasonable, but might still be high depending on the model and dataset size. You could try 2e-5 or 1e-5 for more stability, especially if loss spikes continue.

Batch Size :
Small batches increase noise in loss, but are memory efficient. If your hardware allows, trying batch size 8 or 16 could smooth training.

Epochs:
If loss continues to fluctuate around the same range by Epoch 3, consider training for 1–2 more epochs or using early stopping based on an evaluation metric.

Additional Refinement Ideas:


1.   DPO training benefits from metrics like reward accuracy, win rate, or agreement rate with the preference labels.
Exercise: Pass this to DPOTrainer(..., compute_metrics=compute_metrics).
2.   If ref_model=None, you’re assuming the initial model is its own reference. This is okay for fast experiments, but including a frozen reference model helps in stabilizing the reward differences.
3. For faster convergence try: `dpo_config.lr_scheduler_type = "cosine"  # or "cosine_with_restarts"`





#### Inference

In [27]:
import torch

# Ensure the model is on the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Prepare the input prompt and move it to the same device
input_prompt = "What are the benefits of renewable energy?"
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True, padding=True)

# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference with the model
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**inputs)  # Forward pass to get logits

# Get the logits from the model output
logits = outputs.logits

# Since logits are for each token in the sequence, let's get the logits for the last token
last_token_logits = logits[0, -1, :]  # Shape: (vocab_size,)

# If you're comparing the two responses (chosen and rejected), you'd typically have:
# - The logits for each token in the chosen response
# - The logits for each token in the rejected response

# For simplicity, let’s assume you're comparing two responses, which could be represented by
# selecting the logits corresponding to the "chosen" and "rejected" tokens (if applicable).

# For this example, we'll simply take the logits for two hypothetical tokens (e.g., the first two logits):
chosen_score = last_token_logits[0].item()  # For the chosen response (token index 0)
rejected_score = last_token_logits[1].item()  # For the rejected response (token index 1)

# Compare the scores
if chosen_score > rejected_score:
    final_decision = "The model prefers the chosen response."
else:
    final_decision = "The model prefers the rejected response."

print(final_decision)



The model prefers the chosen response.


In [28]:
from transformers import Trainer, TrainingArguments

# Use the tokenizer to prepare the input prompt and responses
input_prompt = "What are the benefits of renewable energy?"
chosen_response = "Renewable energy reduces greenhouse gas emissions and can create sustainable jobs."
rejected_response = "Renewable energy is expensive and unreliable."

# Preprocess the input data for the trainer (similar to training)
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True, padding=True)

# Create a dataset of the example responses (just for inference in this case)
eval_dataset = [{
    'input_ids': inputs['input_ids'].squeeze(),
    'attention_mask': inputs['attention_mask'].squeeze(),
    'chosen': chosen_response,
    'rejected': rejected_response,
}]

# Now let's use the trainer for inference:
trainer = Trainer(
    model=model,  # Make sure the model is the trained model
    args=dpo_config,  # The same config used during training
)

# We will use the trainer's predict method for inference
predictions = trainer.predict(eval_dataset)

# Now we need to process the predictions and compare logits for the chosen and rejected responses
logits = predictions.predictions

chosen_score = last_token_logits[0].item()  # For the chosen response (token index 0)
rejected_score = last_token_logits[1].item()  # For the rejected response (token index 1)
# Compare the scores to determine the model's decision
if chosen_score > rejected_score:
    final_decision = f"The model chooses the response: '{chosen_response}'"
else:
    final_decision = f"The model chooses the response: '{rejected_response}'"

print(final_decision)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


The model chooses the response: 'Renewable energy reduces greenhouse gas emissions and can create sustainable jobs.'


#### Chosen score > Rejected score, as expected

In [29]:
print(chosen_score)
print(rejected_score)

4.06640625
1.8359375


Note: Use Hugging face login if required.

In [ ]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)